# Projeto 1 - Ciência dos Dados

Nome: Pedro Cliquet Amaral

Nome: Luciano Felix Dias

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Lucius\Documents\Insper\s2_2021.1\CDados\Projeto 1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'League of Legends.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,@aakkari @furia tem um champ no league of lege...,0
1,@raylerchil @noideapraisso @paulokogos_ofc kkk...,0
2,a faculdade ta atrapalhando meu desempenho no ...,1
3,league of legends é pior do que perder q vida ...,1
4,eu te odeio league of legends lixo de jogo lixo,1


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,uma call do supermercado vendo anime e a outra...,0
1,#pulgapreta #leagueoflegends #nimotv pulgapret...,0
2,"fico de cara que vem uns zé buceta fala ""ah ma...",0
3,@hastadvalorant esquece league of legends e ba...,0
4,saudade de jogar league of legends com elas,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Para classificar a relevância consideramos elogios, convites para jogar, críticas e formas de engajamento. Os tweets relevantes são: os elogios, críticas e convites para jogar. Já os irrelevantes são: engajamento em que a pessoa utiliza o produto para auto-divulgação e comentários com conteúdo do jogo.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Cleaner

In [6]:
def normalize (text):
    """
        Função que normaliza os caracteres dos tweets.
    """
    text_lower = text.lower()

    punctuation = "\@\w+\s?|[!-.:?;]"
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text_lower)

    return text_subbed


### Dicionário de palavras

In [7]:
def extract_words (series):
    """
        Função que extrai as palavras de uma Series.
    """
    split = [line.split() for line in series]
    words = [word for line in split for word in line]
    
    return pd.Series(words)


### Naive-Baies

In [8]:
def P_palavra (palavra, dicio, length):
    """
        Função que calcula a probabilidade de uma palavra dado um dicionário de palavras com suavização por Naive-Bayes.
    """
    ocor = dicio[palavra] if palavra in dicio else 0
    prob = (ocor + 1) / (length + len(dicio))

    return prob


### Classificador

In [9]:
class classificador:
    def __init__ (self, df, data_column, category_column):
        self.columns = (data_column, category_column)

        data = df[data_column]
        words = extract_words(data)
        self.length = len(words)

        categories = df[category_column].cat.categories

        self.categories = {}
        for category in categories:
            cat_filter = df[category_column] == category
            filtered = df.loc[cat_filter]
            data = filtered[data_column]
            words = extract_words(data)

            self.categories[category] = {
                "table": words.value_counts(),
                "length": len(words)
            }
    
    def probabilidades (self, frase):
        normalized = normalize(frase)
        
        P_list = []
        for category, data in self.categories.items():
            table = data["table"]
            length = data["length"]
            
            P_cat = length / self.length
            P_frase_dado_cat = 1

            for palavra in normalized.split():
                P = P_palavra(palavra, table, length)
                P_frase_dado_cat *= P

            P_cat_dado_frase = P_frase_dado_cat * P_cat
            P_list.append((category, P_cat_dado_frase))

        return pd.DataFrame(P_list, columns=self.columns)

    def classifica (self, frase):
        sorted_prob = self.probabilidades(frase).sort_values(by="Relevancia", ascending=False)

        return sorted_prob.iloc[0, 0]


### Criando variáveis

In [10]:
# Normalização dos tweets
train["Treinamento"] = train["Treinamento"].apply(normalize)

# Criação das categorias de relevância
train.Relevancia = train.Relevancia.astype('category')
train.Relevancia.cat.categories = (['Irrelevante','Relevante'])

train

,Treinamento,Relevancia
0,tem um champ no league of legends que se chama...,Irrelevante
1,kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk...,Irrelevante
2,a faculdade ta atrapalhando meu desempenho no ...,Relevante
3,league of legends é pior do que perder q vida ...,Relevante
4,eu te odeio league of legends lixo de jogo lixo,Relevante
...,...,...
295,eu não sei pq eu continuo jogando esse jogo de...,Relevante
296,larguei o league of legends vou voltar virar g...,Relevante
297,eu odeio league of legends,Relevante
298,esse league of legends é uma puta pediu pra at...,Relevante


In [11]:
relevante = train['Relevancia'] == 'Relevante'
train_R = train.loc[relevante]

train_R.head()

,Treinamento,Relevancia
2,a faculdade ta atrapalhando meu desempenho no ...,Relevante
3,league of legends é pior do que perder q vida ...,Relevante
4,eu te odeio league of legends lixo de jogo lixo,Relevante
7,league of legends é um jogo podre nojento,Relevante
8,as vezes eu acho que o fato de não poder jogar...,Relevante


In [12]:
irrelevante = train['Relevancia'] == 'Irrelevante'
train_I = train.loc[irrelevante]

train_I.head()

,Treinamento,Relevancia
0,tem um champ no league of legends que se chama...,Irrelevante
1,kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk...,Irrelevante
5,o unico defeito de kim heechul eh ser diamante...,Irrelevante
6,league of legends 4 segredos para chegar ao me...,Irrelevante
10,oi tô ao vivo jogando league of legends https/...,Irrelevante


In [13]:
classificador_tweets_lol = classificador(train, "Treinamento", "Relevancia")

In [14]:
frase = "lalalal bibi tubss"

P_frase = classificador_tweets_lol.probabilidades(frase)

P_frase.sort_values(by="Relevancia", ascending=False).head()

,Treinamento,Relevancia
1,Relevante,2.114103e-11
0,Irrelevante,5.880212e-12


### Teste

In [15]:
test.Relevancia = test.Relevancia.astype('category')
test.Relevancia.cat.categories = (['Irrelevante','Relevante'])

test['Teste'] = test['Teste'].apply(normalize)
test

,Teste,Relevancia
0,uma call do supermercado vendo anime e a outra...,Irrelevante
1,pulgapreta leagueoflegends nimotv pulgapreta e...,Irrelevante
2,fico de cara que vem uns zé buceta fala ah mas...,Irrelevante
3,esquece league of legends e baixa algum moba d...,Irrelevante
4,saudade de jogar league of legends com elas,Relevante
...,...,...
195,depois do assalto em valença que levaram meu c...,Relevante
196,o time de league of legends da furb um dos lin...,Irrelevante
197,mal vejo a hora de chegar em casa pra jogar aq...,Relevante
198,como é bom xingar no jogo league of legends,Relevante


### P ser relevante ou irrelevante


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [16]:
test['verifica'] = test['Teste'].apply(classificador_tweets_lol.classifica)
test

,Teste,Relevancia,verifica
0,uma call do supermercado vendo anime e a outra...,Irrelevante,Relevante
1,pulgapreta leagueoflegends nimotv pulgapreta e...,Irrelevante,Irrelevante
2,fico de cara que vem uns zé buceta fala ah mas...,Irrelevante,Relevante
3,esquece league of legends e baixa algum moba d...,Irrelevante,Relevante
4,saudade de jogar league of legends com elas,Relevante,Relevante
...,...,...,...
195,depois do assalto em valença que levaram meu c...,Relevante,Relevante
196,o time de league of legends da furb um dos lin...,Irrelevante,Relevante
197,mal vejo a hora de chegar em casa pra jogar aq...,Relevante,Relevante
198,como é bom xingar no jogo league of legends,Relevante,Relevante


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**